## Step 2 - Database Engineering

Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.

* Create a Jupyter Notebook called `database_engineering.ipynb` and use this to complete all of your Database Engineering work.

* Use Pandas to read your cleaned measurements and stations CSV data.

* Use the `engine` and connection string to create a database called `hawaii.sqlite`.

* Use `declarative_base` and create ORM classes for each table.

  * You will need a class for `Measurement` and for `Station`.

  * Make sure to define your primary keys.

* Once you have your ORM classes defined, create the tables in the database using `create_all`.

In [7]:
# import dependencies
import pandas as pd
import datetime

In [8]:
# read-in the csvs
hawaii_measurements = pd.read_csv('Resources/clean_hawaii_measurements.csv')
hawaii_measurements['date']=hawaii_measurements['date'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))
hawaii_stations = pd.read_csv('Resources/hawaii_stations.csv')
hawaii_measurements.drop(['Unnamed: 0'], axis=1,inplace=True)

In [9]:
display(hawaii_stations.head())

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [10]:
display(hawaii_measurements.head())

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [11]:
from sqlalchemy import create_engine

#engine = create_engine("sqlite:///some.db") # create sqlite db. use "////" to specify absolute path
engine = create_engine("sqlite:///hawaii.sqlite")

In [12]:
# The declarative system is normally used to configure object relational mapping.

# Creates type Base, it is registry for set of classes (metadata) that descent from Base
# Usually it is enough to have one Base for all types, but it is no problem to work with multiple Bases
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [13]:
# Basic mapping
from sqlalchemy import Column, Integer, String, Float, DateTime

class Stations(Base):
    __tablename__ = "stations"

    station = Column(String,primary_key=True)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
 
from sqlalchemy import ForeignKey

class Measurements(Base):
    __tablename__ = "measurements"

    id = Column(Integer,primary_key=True)
    station = Column(String,ForeignKey("stations.station"))
    date = Column(DateTime)
    prcp = Column(Float)
    tobs = Column(Integer)

In [14]:
# Use `create_all` to create all the table in the database
Base.metadata.create_all(engine)

In [15]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
### BEGIN SOLUTION
data_stations = hawaii_stations.to_dict(orient='records')
data_stations[0]
### END SOLUTION

{'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [16]:
data_measurements = hawaii_measurements.to_dict(orient='records')
data_measurements[1]

{'date': Timestamp('2010-01-02 00:00:00'),
 'prcp': 0.0,
 'station': 'USC00519397',
 'tobs': 63}

In [17]:
# inspect the datatabase with the inspect object

In [18]:
from sqlalchemy import create_engine, inspect

In [19]:
# Create the inspector and connect it to the engine
### BEGIN SOLUTION
inspector = inspect(engine)
### END SOLUTION

In [20]:
# Collect the names of tables within the database
### BEGIN SOLUTION
inspector.get_table_names()
### END SOLUTION

['measurements', 'stations']

In [21]:
# Using the inspector to print the column names within the 'Salaries' table and its types
### BEGIN SOLUTION
columns = inspector.get_columns('stations')
for column in columns:
    print(column["name"], column["type"])
### END SOLUTION

station VARCHAR
name VARCHAR
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [22]:
# Using the inspector to print the column names within the 'Salaries' table and its types
### BEGIN SOLUTION
columns = inspector.get_columns('measurements')
for column in columns:
    print(column["name"], column["type"])
### END SOLUTION

id INTEGER
station VARCHAR
date DATETIME
prcp FLOAT
tobs INTEGER


## load the data into the db

In [24]:
from sqlalchemy import MetaData

metadata = MetaData(bind=engine)
metadata.reflect()

In [26]:
# Save the reference to the `stations` table as a variable called `table`
### BEGIN SOLUTION

import sqlalchemy
table = sqlalchemy.Table('stations', metadata, autoload=True)

In [27]:
conn = engine.connect()
conn.execute(table.delete())
conn.execute(table.insert(), data_stations)

In [28]:
conn.execute("select * from stations").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [29]:
# Save the reference to the `measurements` table as a variable called `table`
### BEGIN SOLUTION
table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [30]:
conn = engine.connect()
conn.execute(table.delete())
conn.execute(table.insert(), data_measurements)

In [31]:
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01 00:00:00.000000', 0.08, 65),
 (2, 'USC00519397', '2010-01-02 00:00:00.000000', 0.0, 63),
 (3, 'USC00519397', '2010-01-03 00:00:00.000000', 0.0, 74),
 (4, 'USC00519397', '2010-01-04 00:00:00.000000', 0.0, 76),
 (5, 'USC00519397', '2010-01-07 00:00:00.000000', 0.06, 70)]